In [ ]:
pip install pydynpd

In [ ]:
import pandas as pd
from  pydynpd import regression
from google.colab import drive

Accessing drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Reading data from the excel file

In [ ]:
dfr = pd.read_excel(r'/content/drive/My Drive/CMIE_PROJECT/panel.xlsx', sheet_name='mainpanel')

Dropping irrelevant variables

In [ ]:
dfr = dfr.drop(columns=['Total assets','Net fixed assets','Export / Sales (%)','IT/ITES & other professional services','Additions to computers and IT systems during the year','Gross software','Net profit margin','Net cash inflow or (outflow) from investing activities','Size','Total forex earnings / Total income (%)','Raw material imports / Raw material purchases (%)', 'Gross computers and IT systems'])


In [ ]:
df=dfr.copy()

In [ ]:
df.columns = df.columns.str.replace(' ', '_')

Filling missing values, linearly

In [ ]:
def fill_nan_linearly(column):
    return column.interpolate(method='linear', limit_direction='both')

filled_df = df.groupby('Companies').apply(lambda group: group.apply(fill_nan_linearly, axis=0))
filled_df.reset_index(drop=True, inplace=True)

Creating Dummies of Year

In [ ]:
df=pd.get_dummies(df, columns=['Year'])
df['Year']=dfr['Year']

ModelL: Lag2 and Lag3 of Total income. Total_expenses, Computer_IT,_net_addition_in_year as IV. Time dummies included

In [ ]:
command_str='Total_income L(1:?).Total_income Computer_IT,_net_addition_in_year Net_computers_and_IT_systems Net_plant_&_machinery,_computers_and_electrical_installations Net_software Profit_after_tax Total_expenses | gmm(Total_income, 2:3) gmm(Total_expenses, 1:.) iv(Computer_IT,_net_addition_in_year) | timedumm'

In [ ]:
mydpd = regression.abond(command_str, df, ['Companies', 'Year'])

 m1
 Dynamic panel-data estimation, two-step system GMM
 Group variable: Companies                         Number of obs = 47944    
 Time variable: Year                               Min obs per group: 0     
 Number of instruments = 156                       Max obs per group: 12    
 Number of groups = 4009                           Avg obs per group: 11.96 
+---------------------------------------------------------------+------------+---------------------+------------+-----------+-----+
|                          Total_income                         |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+---------------------------------------------------------------+------------+---------------------+------------+-----------+-----+
|                        L1.Total_income                        | 0.0471605  |      0.0285738      | 1.6504816  | 0.0988445 |     |
|               Computer_IT,_net_addition_in_year               | 0.0636591  |      0.1015073      | 0.62713